<a href="https://colab.research.google.com/github/Thanatipz/BSC_DPDM23/blob/main/SPI4%E0%B8%A3%E0%B8%B0%E0%B8%A2%E0%B8%B0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [11]:
import numpy as np
import scipy.stats as stats
import pandas as pd
from google.colab import drive

# เชื่อมต่อ Google Drive
drive.mount('/content/drive')

# ฟังก์ชันฟิต Gamma Distribution
def fit_gamma_distribution(precipitation):
    precipitation = np.array(precipitation)
    precipitation = precipitation[precipitation > 0]
    if len(precipitation) == 0:
        raise ValueError("ไม่มีข้อมูลที่มากกว่า 0 สำหรับฟิต Gamma Distribution")
    shape, loc, scale = stats.gamma.fit(precipitation, floc=0)
    return shape, scale

# ฟังก์ชันคำนวณ CDF ของ Gamma Distribution
def gamma_cdf(x, shape, scale):
    return stats.gamma.cdf(x, shape, scale=scale)

# ฟังก์ชันคำนวณ SPI
def spi(precipitation):
    precipitation = np.array(precipitation)
    if np.any(precipitation <= 0):
        print("พบค่าที่เป็น 0 หรือค่าลบในข้อมูลฝน ใช้ Gaussian Distribution แทน")
        spi_values = stats.zscore(precipitation)
    else:
        shape, scale_param = fit_gamma_distribution(precipitation)
        cdf_vals = gamma_cdf(precipitation, shape, scale_param)
        spi_values = stats.norm.ppf(cdf_vals)
    return spi_values

# ฟังก์ชันแปลงวันที่จาก YYYYDDD เป็นรูปแบบวันที่ปกติ
def convert_yyyyddd_to_date(yyyyddd):
    yyyyddd = int(yyyyddd)
    year = yyyyddd // 1000
    day_of_year = yyyyddd % 1000
    return pd.to_datetime(f"{year}-{day_of_year}", format="%Y-%j")

# ฟังก์ชันอ่านและคำนวณค่า SPI
def read_and_aggregate_excel(file_path, sheet_name=0):
    df = pd.read_excel(file_path, sheet_name=sheet_name)
    df.columns = ["Date", "Rainfall"]
    df.dropna(subset=["Date", "Rainfall"], inplace=True)
    df["Date"] = df["Date"].apply(convert_yyyyddd_to_date)
    df.set_index("Date", inplace=True)

    periods = []
    interval_days = [35, 15, 20, 20]
    spi_columns = {"35 Days": [], "15 Days": [], "20 Days (1)": [], "20 Days (2)": [], "Total": []}

    for year in range(df.index.year.min(), df.index.year.max()):
        start_date = pd.Timestamp(year=year, month=11, day=1)
        end_date = pd.Timestamp(year=year + 1, month=1, day=31)

        if start_date not in df.index or end_date not in df.index:
            continue  # ข้ามปีที่ไม่มีข้อมูล

        df_subset = df.loc[start_date:end_date]

        if len(df_subset) < sum(interval_days):
            continue  # ข้ามปีที่มีข้อมูลไม่ครบ

        total_rainfall = 0
        period_data = []
        date_pointer = start_date

        for idx, days in enumerate(interval_days):
            period_end = date_pointer + pd.DateOffset(days=days - 1)
            rainfall_sum = df_subset.loc[date_pointer:period_end, "Rainfall"].sum()
            period_data.append(rainfall_sum)

            # บันทึกค่า SPI ของแต่ละช่วง
            if idx == 0:
                spi_columns["35 Days"].append(rainfall_sum)
            elif idx == 1:
                spi_columns["15 Days"].append(rainfall_sum)
            elif idx == 2:
                spi_columns["20 Days (1)"].append(rainfall_sum)
            elif idx == 3:
                spi_columns["20 Days (2)"].append(rainfall_sum)

            total_rainfall += rainfall_sum
            date_pointer = period_end + pd.DateOffset(days=1)

        spi_columns["Total"].append(total_rainfall)
        periods.append((start_date, end_date, total_rainfall))

    period_df = pd.DataFrame(periods, columns=["Start Date", "End Date", "Total Rainfall"])

    # คำนวณค่า SPI ของแต่ละช่วง
    for key in spi_columns:
        if len(spi_columns[key]) == len(period_df):
            period_df[f"SPI {key}"] = spi(spi_columns[key])

    return period_df

# อ่านไฟล์และคำนวณ SPI
file_path = "/content/drive/MyDrive/SPI/rain.xlsx"
precip_data = read_and_aggregate_excel(file_path)

# แปลงวันที่เป็น string เพื่อแสดงผล
spi_df = precip_data.copy()
spi_df["Start Date"] = spi_df["Start Date"].dt.strftime("%Y-%m-%d")
spi_df["End Date"] = spi_df["End Date"].dt.strftime("%Y-%m-%d")

# แสดง DataFrame
print(spi_df)

# บันทึกลงไฟล์ Excel
output_path = "/content/drive/MyDrive/SPI/spi1234_results.xlsx"
spi_df.to_excel(output_path, index=False)
print(f"บันทึกไฟล์ Excel ที่: {output_path}")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
พบค่าที่เป็น 0 หรือค่าลบในข้อมูลฝน ใช้ Gaussian Distribution แทน
พบค่าที่เป็น 0 หรือค่าลบในข้อมูลฝน ใช้ Gaussian Distribution แทน
พบค่าที่เป็น 0 หรือค่าลบในข้อมูลฝน ใช้ Gaussian Distribution แทน
พบค่าที่เป็น 0 หรือค่าลบในข้อมูลฝน ใช้ Gaussian Distribution แทน
    Start Date    End Date  Total Rainfall  SPI 35 Days  SPI 15 Days  \
0   2003-11-01  2004-01-31             7.9    -1.015208    -0.432703   
1   2004-11-01  2005-01-31             2.8    -0.937275    -0.432703   
2   2005-11-01  2006-01-31            43.7     1.187699    -0.432703   
3   2006-11-01  2007-01-31             6.8    -0.661911    -0.432703   
4   2007-11-01  2008-01-31             4.6    -0.874928    -0.405743   
5   2008-11-01  2009-01-31            35.6     0.818816    -0.432703   
6   2009-11-01  2010-01-31            39.3    -0.599565    -0.432703   
7   2010-11-01  2011-01-31         